In [1]:
# calling the libraries 
import cv2
import numpy as np
import matplotlib.pyplot as plt


# for detecting the edges in the image
def canny(image):
    # converting the image from RGB to Gray 
    gray=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    # blurring the image with the gassian filter of 5*5 filter
    blur = cv2.GaussianBlur(gray,(5,5),0)
    # cv2.canny(image ,lower_threshold ,Upper_threshold).if gradient is above the threshold it is accepted as edge pixel
    # if the threshold is in between upper and lower then edge is only accepted if it is align to strong edge 
    canny=cv2.Canny(blur,50,150)
    return canny

def make_coordinates(image,line_parameters):
    slope,intercept = line_parameters
    """ Here we want to make line from origin of the image dimensions (H-704,W-1279,C-3).So, y-axis starts from 704(bottom) to 0(top) 
    till the y2 (i:e- 422.4) .
    """ 
    y1=image.shape[0]
    y2=int(y1*(3/5))
    x1=int((y1-intercept)/slope)
    x2=int((y2-intercept)/slope)
    # print(image.shape)
    return np.array([x1,y1,x2,y2])
""" During the lines detection we got mutiple line which crosses the the  threshold.
So in order to make a single line we would Average the lines. """ 
def averaged_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)
    try:
        left_line = make_coordinates(image, left_fit_average)
        right_line = make_coordinates(image, right_fit_average)
        return np.array([left_line, right_line])
    except Exception as e:
#         print(e, '\n')
 #print error to console
        return None


def display_lines(image,lines):# function for displaying line in image
    #making line image of the same size as of the image as of black
    line_image=np.zeros_like(image)
    # checking if the lines is not empty
    if lines is not None:
        for line in lines:
            #print(line)--- output would be a line with 2-D array
            x1,y1,x2,y2=line.reshape(4)
            # converting 2-D array into sigle line
            # Draws line segment connecting two point ,(B,G,R) colour of the line,line thickness
            #cv2.line(On which image,(co-ordinates),(255,0,0),10)
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    return line_image

def region_of_interest(image):
        # y-axis value 
        height = image.shape[0]
        polygons = np.array([
        [(200,height ), (1100,height), (550,250)]
        ])
        # This will create a mask of same size and dimensions with all zero intensity pixel(black)
        mask= np.zeros_like(image)
        # This will create a polygon on  mask with the intensity of 255.
        cv2.fillPoly(mask, polygons, 255)
        # bitwise and operation of the image and mask so the areas under the mask which is 1 is overlapped with the image
        # Inshort result would be the canny edge under the masked region
        masked_image=cv2.bitwise_and(image,mask)
        
        return masked_image

    

  
# video capture object
cap =cv2.VideoCapture("Lane_marking.mp4")
    
while (cap.isOpened()):
    _, frame=cap.read()
    canny_image=canny(frame)
    cropped_image=region_of_interest(canny_image)
    # hough tranfrom (image,value of rho,value of the theta,threshold ,line less then 40 pixel are regeted,the max distance between the two pixel lines would be 5 means if there two lines with a gap of 5 are connected to one line instead of broken up) )
    lines=cv2.HoughLinesP(cropped_image,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=5)
    averaged_lines = averaged_slope_intercept(frame,lines)
    line_image=display_lines(frame,averaged_lines)
    # Adding of the two image with the weights (intensity color)specified next to it. 
    # combo_image=cv2.addWeighted(image_1,0.8,image_2,1,scalar_value to the addition of the image)
    combo_image=cv2.addWeighted(frame,0.8,line_image,1,1)
    cv2.imshow('results', combo_image)
    # waitkey each frame will wait for 1 milli-sec
     
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
# releasing the video file        
cap.release()

cv2.destroyAllWindows()

/home/shujat/.virtualenvs/py3/lib/python3.5/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/shujat/.virtualenvs/py3/lib/python3.5/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# Output of Lane_Detection:


<img src="Results_Lane_detection.png" width="400" height="200">